In [1]:
from gym_citynav.utils.unrealcv_basic import UnrealCV
from gym_citynav.utils.unrealcv_basic_with_traffic import UnrealCV
from gym_citynav.utils.environment_generator_with_traffic import EnvironmentGenerator

d:\anaconda\envs\CityNav\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unrealcv = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1280, 960))
# resolution = (1280, 900)
# # time.sleep(1)
# unrealcv.client.request("vset /camera/1/size {} {}".format(resolution[0], resolution[1]))
# unrealcv.client.request("vset /camera/1/size {} {}".format(resolution[0], resolution[1]))
# unrealcv.client.request("vset /camera/1/reflection Lumen")
# unrealcv.client.request("vset /camera/1/illumination Lumen")
# unrealcv.client.request("vset /camera/1/exposure_bias 4.5")

INFO:__init__:230:Got connection confirm: b'connected to gym_citynav'


=>Info: using ip-port socket


In [6]:
import os
import cv2
import time
def save(frames, output_path, start_frame):
    """
    Save images
    """
    current_frame = start_frame
    for frame in frames:
        image = frame
        path = os.path.join(os.getcwd(), output_path, f"{current_frame}.png")
        # print("save images to ", path)
        os.makedirs(os.path.dirname(path), exist_ok=True)
        # save image to folder
        cv2.imwrite(path, image)
        current_frame += 1

In [7]:
unrealcv.get_cameras()

['PawnSensor', 'FusionCamSensor']

In [17]:
# time.sleep(6)
fps = 30
frames = []
camera_num = 1
resolution = (1920, 1080)
# time.sleep(1)
unrealcv.client.request("vset /camera/{}/size {} {}".format(camera_num, resolution[0], resolution[1]))
unrealcv.client.request("vset /camera/{}/reflection Lumen".format(camera_num))
unrealcv.client.request("vset /camera/{}/illumination Lumen".format(camera_num))
unrealcv.client.request("vset /camera/{}/exposure_bias 4.5".format(camera_num))
# unrealcv.client.request("vset /camera/1/exposure_bias 4.5")
while True:
    unrealcv.client.request("vset /camera/{}/exposure_bias 0".format(camera_num))
    unrealcv.client.request("vset /camera/{}/exposure_bias 6".format(camera_num))
    img = unrealcv.read_image("1", "lit")
    frames.append(img)
    # time.sleep(1/fps)
# cv2.imshow("img", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

KeyboardInterrupt: 

In [15]:
len(frames)

222

In [18]:
# save(frames, "output_videos/images/single_agent/orientation", 0)
# save(frames, "output_videos/images/single_agent/move_along_road", 0)
# save(frames, "output_videos/images/single_agent/turning", 0)
save(frames, "output_videos/images/different_agents/meta_human_run", 0)


In [19]:
rename_frames("output_videos/images/different_agents/meta_human_run")
render_video("output_videos/images/different_agents/meta_human_run", output="different_agents_meta_human_run.mp4")

✅ Renamed 226 files in 'output_videos\images\different_agents\meta_human_run'
🎬 Rendering video: different_agents_meta_human_run.mp4
✅ Video rendering complete.


In [ ]:
rename_frames("output_videos/images/single_agent/orientation")
rename_frames("output_videos/images/single_agent/move_along_road")
rename_frames("output_videos/images/single_agent/turning")
rename_frames("output_videos/images/single_agent/find_destination")
render_video("output_videos/images/single_agent/orientation", output="orentation.mp4")
render_video("output_videos/images/single_agent/move_along_road", output="move_along_road.mp4")
render_video("output_videos/images/single_agent/turning", output="turning.mp4")
render_video("output_videos/images/single_agent/find_destination", output="find_destination.mp4")

In [ ]:
def save_video_from_images(image_path, video_path, fps=10):
    import imageio
    import os

    # 图像文件夹路径
    image_folder = image_path
    # 视频输出路径
    output_video = video_path

    # 获取所有图像文件并按文件名数字顺序排序
    images = os.listdir(image_folder)
    images.sort(key=lambda x: int(x.split('.')[0]))
    print(images)
    images = [os.path.join(image_folder, image) for image in images]

    # 创建一个视频写入对象
    with imageio.get_writer(output_video, fps=fps) as writer:
        for image_name in images:
            image_path = os.path.join(image_folder, image_name)
            image = imageio.imread(image_path)  # 读取图像
            writer.append_data(image)  # 将图像写入视频

In [3]:
import os
import subprocess
from pathlib import Path

def rename_frames(input_dir, prefix="frame_", ext="png"):
    """
    Rename all image files in the input directory to the format: prefix_0001.ext, prefix_0002.ext, ...
    
    Args:
        input_dir (str or Path): Path to the folder containing the images.
        prefix (str): Prefix for renamed files (default: 'frame_').
        ext (str): Extension of the image files (default: 'png').
    """
    input_dir = Path(input_dir)
    if not input_dir.exists():
        raise FileNotFoundError(f"Directory not found: {input_dir}")
    
    # 获取所有目标文件
    files = sorted(input_dir.glob(f"*.{ext}"), key=lambda f: int(f.stem))
    
    for i, file in enumerate(files, start=1):
        new_name = input_dir / f"{prefix}{i:04d}.{ext}"
        file.rename(new_name)
    
    print(f"✅ Renamed {len(files)} files in '{input_dir}'")

def render_video(input_dir, prefix="frame_", ext="png", fps=30, output="output.mp4"):
    """
    Use FFmpeg to render a video from image frames.

    Args:
        input_dir (str or Path): Folder containing renamed frames.
        prefix (str): Prefix of frame files.
        ext (str): File extension (default: 'png').
        fps (int): Frames per second for the video.
        output (str): Output video filename.
    """
    input_dir = Path(input_dir)
    input_pattern = str(input_dir / f"{prefix}%04d.{ext}")

    cmd = [
        "ffmpeg",
        "-framerate", str(fps),
        "-i", input_pattern,
        "-s", "1280x900",
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        output
    ]

    print(f"🎬 Rendering video: {output}")
    subprocess.run(cmd, check=True)
    print("✅ Video rendering complete.")


orientation_task 
(-6972.0, 7756.0， 58) (0, 0, 90)
move along road task
(-6972.0, 7756.0， 58) (0, 0, 0)
Turning Task:
(-3971.999508, 7756.0， 63.056413) (0,0,0)
Find Destination:
(-1346.999078, 7756.0, 59.945384) (0, 0, 90)
Destination:
(-1346.999078, 10206.000402, 59.945384) (0, 0, 0)


beginning location
(-3499.0, 7860.0， 60.0) (0， 0， 0)




In [6]:
import pandas as pd

df = pd.read_csv(r'D:\github\SimWorld\case study\single\trajectory.csv')
column_data = df['action']
values = column_data.dropna().tolist()
filtered = [x for x in values if x != -1.0]


In [9]:
# unrealcv = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1280, 960))
world_json = r"D:\github\SimWorld\case study\single\task_dist_11_0_1\progen_world.json"
road_json = r"D:\github\SimWorld\case study\single\task_dist_11_0_1\roads.json"
# unrealcv = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1920, 1080))
world_generator = EnvironmentGenerator(unrealcv)
# Clean the environment
world_generator.clear_env()
# world_generator = EnvironmentGenerator(unrealcv)
# # Clean the environment
# world_generator.clear_env()
# Loading Json
world_generator.loading_json(world_json)
# Generate World
world_generator.generate_world()

In [ ]:
robot_name = "spot_robot" 
unrealcv.spawn_bp_asset("/Game/Robot_Dog/Blueprint/BP_SpotRobot.BP_SpotRobot_C", robot_name)
camera_num = 1
unrealcv.client.request("vset /camera/{}/reflection Lumen".format(camera_num))
unrealcv.client.request("vset /camera/{}/illumination Lumen".format(camera_num))
# unrealcv.client.request("vset /camera/{}/exposure_bias 4.5".format(camera_num))
resolution = (1280, 900)
# time.sleep(1)
unrealcv.client.request("vset /camera/{}/size {} {}".format(camera_num, resolution[0], resolution[1]))

In [11]:
unrealcv.enable_controller(robot_name, True)
unrealcv.set_location_hard([-45154.5228, -61700.0, 73.957685], robot_name)
unrealcv.set_orientation_hard([0, 0, 90], robot_name)

In [12]:
import time
fps = 30
frames = []
camera_num = 1
# test_actions = [5.0, 0.0, 0.0]
for action in filtered:
    if action == 4.0:
        unrealcv.apply_action_rotation(robot_name, [0.5, 90, 1])
    elif action == 5.0:
        unrealcv.apply_action_rotation(robot_name, [0.5, -90, -1])
    elif action == 0.0:
        unrealcv.apply_action_transition(robot_name, [500, 1, 0])
    available = False
    while not available:
        # unrealcv.client.request("vset /camera/{}/exposure_bias 0".format(camera_num))
        # unrealcv.client.request("vset /camera/{}/exposure_bias 6".format(camera_num))
        img = unrealcv.read_image("1", "lit")
        frames.append(img)
        time.sleep(1/fps)
        available = unrealcv.get_is_available(robot_name)
        # print(available)

In [15]:
save(frames, "output_videos/images/single_agent_qualitive/", 0)

In [ ]:
# rename_frames("output_videos/images/single_agent_qualitive")
render_video("output_videos/images/single_agent_qualitive", output="case_1_updated.mp4")

In [ ]:
import cv2
import os

# 设置路径
input_folder = r'D:\github\SimWorld\output_videos\images\images2litup'      # 输入图片所在文件夹
output_folder = r'D:\github\SimWorld\output_videos\images\brightened' # 输出保存的新图片文件夹
os.makedirs(output_folder, exist_ok=True)

# 调整亮度的参数
alpha = 1.0  # 对比度（1.0 表示不变）
beta = 20    # 亮度（增加数值会变亮）

# 遍历文件夹中的所有图片
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"跳过无效图片: {filename}")
            continue

        # 提高亮度
        bright_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        # 保存结果
        out_path = os.path.join(output_folder, filename)
        cv2.imwrite(out_path, bright_img)

print("所有图片亮度已调高并保存到输出文件夹。")


In [48]:
# unrealcv = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1280, 960))
world_json = r"D:\github\SimWorld\case study\multi\task_dist_1_1\progen_world.json"
road_json = r"D:\github\SimWorld\case study\multi\task_dist_1_1\roads.json"
# unrealcv = UnrealCV(port=9000, ip='127.0.0.1', resolution=(1920, 1080))
world_generator = EnvironmentGenerator(unrealcv)
# Clean the environment
world_generator.clear_env()
# world_generator = EnvironmentGenerator(unrealcv)
# # Clean the environment
# world_generator.clear_env()
# Loading Json
world_generator.loading_json(world_json)
# Generate World
world_generator.generate_world()

In [49]:
import pandas as pd

df = pd.read_csv(r'D:\github\SimWorld\case study\multi\multi-success-case-main-trajectory.csv')
column_data = df['action']
values = column_data.dropna().tolist()
main_agent_actions = [x for x in values if x != 100]

df = pd.read_csv(r'D:\github\SimWorld\case study\multi\multi-success-case-follower-trajectory.csv')
column_data = df['action']
values = column_data.dropna().tolist()
follower_agent_actions = [x for x in values if x != 100]

In [50]:
len(main_agent_actions)

32

In [51]:
len(follower_agent_actions)

45

In [52]:
robot_name_1 = "spot_robot_1" 
unrealcv.spawn_bp_asset("/Game/Robot_Dog/Blueprint/BP_SpotRobot.BP_SpotRobot_C", robot_name_1)
robot_1_camera = 1
unrealcv.client.request("vset /camera/{}/reflection Lumen".format(robot_1_camera))
unrealcv.client.request("vset /camera/{}/illumination Lumen".format(robot_1_camera))
# unrealcv.client.request("vset /camera/{}/exposure_bias 4.5".format(camera_num))
resolution = (1280, 900)
# time.sleep(1)|
unrealcv.client.request("vset /camera/{}/size {} {}".format(robot_1_camera, resolution[0], resolution[1]))

robot_name_2 = "spot_robot_2" 
unrealcv.spawn_bp_asset("/Game/Robot_Dog/Blueprint/BP_SpotRobot.BP_SpotRobot_C", robot_name_2)
robot_2_camera = 2
unrealcv.client.request("vset /camera/{}/reflection Lumen".format(robot_2_camera))
unrealcv.client.request("vset /camera/{}/illumination Lumen".format(robot_2_camera))
# unrealcv.client.request("vset /camera/{}/exposure_bias 4.5".format(camera_num))
resolution = (1280, 900)
# time.sleep(1)
unrealcv.client.request("vset /camera/{}/size {} {}".format(robot_2_camera, resolution[0], resolution[1]))

'ok'

In [56]:
unrealcv.enable_controller(robot_name_1, True)
unrealcv.set_location_hard([34419.4805, -1700.0, 73.957685], robot_name_1)
unrealcv.set_orientation_hard([0, 0, -90], robot_name_1)

unrealcv.enable_controller(robot_name_2, True)
unrealcv.set_location_hard([61700.0, -9406.5421, 73.957685], robot_name_2)
unrealcv.set_orientation_hard([0, 0, 0], robot_name_2)

In [57]:

import time
fps = 30
frames = []
camera_num = 1
# test_actions = [5.0, 0.0, 0.0]

for i in range(len(follower_agent_actions)):
    agent_1_action = 100
    if i < len(main_agent_actions):
        agent_1_action = main_agent_actions[i]
    agent_2_action = follower_agent_actions[i]
    if agent_1_action != 100:
        if agent_1_action == 4.0:
            unrealcv.apply_action_rotation(robot_name_1, [1, 90, 1])
        elif agent_1_action == 5.0:
            unrealcv.apply_action_rotation(robot_name_1, [1, -90, -1])
        elif agent_1_action == 0.0:
            unrealcv.apply_action_transition(robot_name_1, [500, 1, 0])
    
    if agent_2_action == 4.0:
        unrealcv.apply_action_rotation(robot_name_2, [1, 90, 1])
    elif agent_2_action == 5.0:
        unrealcv.apply_action_rotation(robot_name_2, [1, -90, -1])
    elif agent_2_action == 0.0:
        unrealcv.apply_action_transition(robot_name_2, [500, 1, 0])
    available = False
    while not available:
        # # unrealcv.client.request("vset /camera/{}/exposure_bias 0".format(camera_num))
        # # unrealcv.client.request("vset /camera/{}/exposure_bias 6".format(camera_num))
        img = unrealcv.read_image("1", "lit")
        frames.append(img)
        time.sleep(1/fps)
        # time.sleep(0.2)
        available = unrealcv.get_is_available(robot_name_2)

In [58]:
# save(frames, "output_videos/images/multi_agent_qualitive/follower", 0)
save(frames, "output_videos/images/multi_agent_qualitive/main", 0)


In [60]:
import cv2
import os

# 设置路径
input_folder = r'D:\github\SimWorld\output_videos\images\images2litup'      # 输入图片所在文件夹
output_folder = r'D:\github\SimWorld\output_videos\images\brightened' # 输出保存的新图片文件夹
os.makedirs(output_folder, exist_ok=True)

# 调整亮度的参数
alpha = 1.0  # 对比度（1.0 表示不变）
beta = 20    # 亮度（增加数值会变亮）

# 遍历文件夹中的所有图片
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"跳过无效图片: {filename}")
            continue

        # 提高亮度
        bright_img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        # 保存结果
        out_path = os.path.join(output_folder, filename)
        cv2.imwrite(out_path, bright_img)

print("所有图片亮度已调高并保存到输出文件夹。")


所有图片亮度已调高并保存到输出文件夹。


In [63]:
# rename_frames(r"D:\github\SimWorld\output_videos\images\multi_agent_qualitive\follower")
# rename_frames(r"D:\github\SimWorld\output_videos\images\multi_agent_qualitive\main")
# render_video(r"D:\github\SimWorld\output_videos\images\multi_agent_qualitive\follower", output="multi_qualitive_follower.mp4")
render_video(r"D:\github\SimWorld\output_videos\images\multi_agent_qualitive\main", output="multi_qualitive_main.mp4")

🎬 Rendering video: multi_qualitive_main.mp4
✅ Video rendering complete.
